In [ ]:
# 구글 drive 연결
from google.colab import drive
drive.mount("/content/drive")

!pip install --upgrade pip
!pip install --root-user-action=ignore requests
!pip install mecab-python3

# 코랩 연동 konlpy, Mecab 형태소 분석기 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

# gensim 설치
!pip install gensim
from gensim.models import Word2Vec
from konlpy.tag import Okt, Mecab
import pandas as pd

file_path = "/content/drive/MyDrive/NLP/nsmc.txt"                     # 데이터 경로
df = pd.read_table(file_path)                                         # 데이터 읽기
print(df.head())

# 데이터 전처리
pre_null = df.isnull().sum()                                          #데이터 결측치 확인
post_null = df.dropna()                                               # 결측치 데이터 제거하기
print(post_null.isnull().sum())

df_documemt = post_null.document.str.replace('[^가-힣 ]', '')         # 한글+공백 외 문자 제거
print(df_documemt)

stopwords = ['의', '가', '이', '들', '은', '을', '를', '는',          # 불용어 지정
             '외', '좀', '잘', '걍', '과', '와', '도',
             '처럼', '민큼', '보다', '하고', '라고', '야',
             '아', '으로', '로써', '로서', '자', '에게', '께',
             '에', '에서', '로', '부터', '한테', '때문에',
             '그', '저', '고', '이런', '저런', '무슨',
             '또', '즉', '또는', '혹', '곧', '또한', '까지']

data = df_documemt[:100000]                                           # 일부 데이터 활용
print(data.head())

# Mecab 형태소 분석기
mecab = Mecab()                                                       # Mecab 형태소 분석기 객체 생성

tokenized_data = []
for sentence in data:
    sentence_tokenized = []
    tokenized = mecab.morphs(sentence)                                #토큰화

    for morphs in tokenized:                                          # 불용어 제거
        if morphs not in stopwords:
            sentence_tokenized.append(morphs)
    tokenized_data.append(sentence_tokenized)

print(tokenized_data[:3])                                             # 단어 사전 확인

# Skipgram_Mecab model 응용
model_skipgram = Word2Vec(sentences = tokenized_data,                       # Skipgram_Mecab model 생성
                      vector_size = 100,
                      window= 3,
                      min_count = 5,
                      workers = 1,
                      sg = 1)

skipgram_mecab_path = '/content/drive/MyDrive/NLP/skipgram_mecab_model'         # 모델 저장
model_skipgram.save(skipgram_mecab_path)
model_skipgram_mecab = Word2Vec.load(skipgram_mecab_path)                        # 모델 불러오기

# CBOW model 활용하기
print(f'특정 단어에 대한 word vector: \n{model_skipgram_mecab.wv["단어"]}') #특정 단어의 embedding vector 확인

vocab = model_skipgram_mecab.wv.key_to_index                                # 생성된 단어 사전 확인(키와 인덱스)
print(f'단어사전: \n{vocab}')
print(len(vocab))                                                           # 단어 사전 길이 확인
print(vocab.keys())                                                         # 단어 사전의 전체 key 확인
print(model_skipgram_mecab.wv.vectors)                                      # 단어 사전의 전체 value 확인
print(model_skipgram_mecab.wv.vectors.shape)                                # 임베딩 벡터의 모양 확인

# 유사도 비교
w1 = '@@@'
w2 = '###'
similar = model_skipgram_mecab.wv.similarity(w1, w2)
print(similar)

# 특정 단어와  가까운 단어 추출하기
extract_word = model_skipgram_mecab.wv.most_similar("@@@", topn = 5)
print(extract_word)